In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm


In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


In [3]:
# Train data 
train = torchvision.datasets.CIFAR10('./CIFAR10',
                                     train=True,
                                     download=False,
                                     transform=transform)
# Test data 

test = torchvision.datasets.CIFAR10('./CIFAR10',
                                     train=False,
                                     download=False,
                                     transform=transform)

In [4]:
from torch.utils.data import DataLoader

# Load data into DataLoader
trainloader = DataLoader(train,64,shuffle=True)
testloader = DataLoader(test,64,shuffle=True)

In [5]:
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

In [6]:
# Make a Convolutional neural network
class ConvNet(nn.Module):
    def __init__(self) -> None:
        # Don't forget 
        super().__init__() 
        # Conv layers
        self.pool = nn.MaxPool2d(2, 2)
        self.cl1 = nn.Conv2d(3,32,3) # 3 inputs for rgb
        self.cl2 = nn.Conv2d(32,64,3)
        self.cl3 = nn.Conv2d(64,64,3)
        #Fully connected layers
        self.fc1 = nn.Linear(64 * 4 * 4, 64)
        # After convolutions & pooling, size calculation:
        # Input: 32x32 -> conv1 -> 30x30 -> pool -> 15x15
        # -> conv2 -> 13x13 -> pool -> 6x6
        # -> conv3 -> 4x4 with 64 channels
        self.fc2 = nn.Linear(64, 10)
        self.relu = nn.ReLU()
    def forward(self,x):
        # P for poling
        x = self.pool(self.relu(self.cl1(x)))
        x = self.pool(self.relu(self.cl2(x)))
        x = self.relu(self.cl3(x))
        x = x.view(-1, 64 * 4 * 4)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


    

In [ ]:
# At the start of your script, add this device setup and dont be stupid use gpu:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [ ]:
def train_model(model,loss_function,optimaizer,EPOCHS=10):
    model.train()
    for epoch in range(EPOCHS):
        running_loss = 0.0
        epoch_iterator = tqdm(enumerate(trainloader),total=len(trainloader),desc=f"Epoch {epoch+1}/{EPOCHS}")
        for i,(images,labels) in epoch_iterator:
            images = images.to(device)
            labels = labels.to(device)
            

            optimaizer.zero_grad()

            outputs = model(images)
            loss = loss_function(outputs,labels)
            loss.backward()
            optimaizer.step()

            running_loss += loss.item()
            
            if i % 100 == 99:  # Update every 100 batches
                epoch_iterator.set_postfix(loss=(running_loss / 100))
                running_loss = 0.0

        # Print loss for the current epoch
        print(f'Epoch [{epoch+1}/{EPOCHS}] - Loss: {running_loss / len(trainloader):.3f}')

In [18]:
from tqdm import tqdm

def test_model(model, testloader, device):
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    class_correct = [0] * 10
    class_total = [0] * 10
    
    with torch.no_grad():  # Disable gradient computation for testing
        for images, labels in tqdm(testloader, desc="Testing", leave=False):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            # Per-class accuracy
            c = (predicted == labels)
            for i in range(len(labels)):
                label = labels[i].item()
                class_correct[label] += c[i].item()
                class_total[label] += 1

    # Calculate and print overall accuracy
    overall_accuracy = 100 * correct / total
    print(f'Overall Accuracy: {overall_accuracy:.2f}%')
    
    for i in range(10):
        if class_total[i] > 0:
            print(f'Accuracy of class {i}: {100 * class_correct[i] / class_total[i]:.2f}%')


In [31]:
model = ConvNet().to(device)
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
train_model(model,loss_function=loss,optimaizer=optimizer,)
test_model(model,device=device,testloader=testloader)

Epoch 1/10: 100%|██████████| 782/782 [00:44<00:00, 17.67it/s, loss=1.34]


Epoch [1/10] - Loss: 0.139


Epoch 2/10: 100%|██████████| 782/782 [00:56<00:00, 13.92it/s, loss=1.13]


Epoch [2/10] - Loss: 0.117


Epoch 3/10: 100%|██████████| 782/782 [01:00<00:00, 13.01it/s, loss=1.01] 


...

Epoch 9/10: 100%|██████████| 782/782 [00:50<00:00, 15.60it/s, loss=0.655]


Epoch [9/10] - Loss: 0.067


Epoch 10/10: 100%|██████████| 782/782 [00:54<00:00, 14.23it/s, loss=0.606]


Epoch [10/10] - Loss: 0.065


Overall Accuracy: 71.41%
Accuracy of class 0: 73.50%
Accuracy of class 1: 69.50%
Accuracy of class 2: 54.60%
Accuracy of class 3: 55.80%
Accuracy of class 4: 68.70%
Accuracy of class 5: 68.50%
Accuracy of class 6: 81.00%
Accuracy of class 7: 75.50%
Accuracy of class 8: 81.70%
Accuracy of class 9: 85.30%


In [32]:
net = ConvNet()

In [33]:
torch.save(net, 'model2.pth')

torch.save(net.state_dict(), 'model_weights2.pth')
